The purpose of this notebook is showing that the new NaimaModel class provides sensible fitting results. The fit performed in the [ joint-crab/4_naima.ipynb](https://github.com/open-gamma-ray-astro/joint-crab/blob/master/4_naima.ipynb) notebook is reproduced, with identical results

In [13]:
from gammapy.spectrum import SpectrumObservationList, SpectrumFit
from gammapy.spectrum.models import NaimaModel
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from naima import models, radiative

## Model definition

In [14]:
particle_parameters={'amplitude' : 1e34 / u.eV, 'e_0' : 3 * u.TeV, 'alpha' : 2.7, 'e_cutoff' : 10 * u.TeV}
particle_distribution = models.ExponentialCutoffPowerLaw(**particle_parameters)

radiative_model = radiative.InverseCompton(
            particle_distribution,
            seed_photon_fields=["CMB"],
            Eemin=100 * u.GeV,
)

model = NaimaModel(radiative_model)

Freeze some parameters before fitting:

In [15]:
model.e_0.frozen = True
model.beta.frozen = True

In [16]:
model.parameters.parameters

[Parameter(name='amplitude', value=1e+34, factor=1e+34, scale=1.0, unit='eV-1', min=nan, max=nan, frozen=False),
 Parameter(name='e_0', value=3.0, factor=3.0, scale=1.0, unit='TeV', min=nan, max=nan, frozen=True),
 Parameter(name='alpha', value=2.7, factor=2.7, scale=1.0, unit='', min=nan, max=nan, frozen=False),
 Parameter(name='e_cutoff', value=10.0, factor=10.0, scale=1.0, unit='TeV', min=nan, max=nan, frozen=False),
 Parameter(name='beta', value=1.0, factor=1.0, scale=1.0, unit='', min=nan, max=nan, frozen=True)]

## Load the data 

In [5]:
from joint_crab.conf import config
dataset = config.datasets["joint"]
obs_list = dataset.get_SpectrumObservationList()
print(f"Number of SpectrumObservation: {len(obs_list)}")

Number of SpectrumObservation: 10


## Fit

In [6]:
fit = SpectrumFit(obs_list=obs_list, model=model, fit_range=dataset.energy_range, stat='wstat')
result = fit.run()

In [7]:
print(result)

OptimizeResult

	backend    : minuit
	method     : minuit
	success    : True
	nfev       : 187
	total stat : 460.27
	message    : Optimization terminated successfully.



In [8]:
print(fit.result[0])


Fit result info 
--------------- 
Model: NaimaModel

Parameters: 

	   name     value     error   unit min max frozen
	--------- --------- --------- ---- --- --- ------
	amplitude 3.026e+34 1.228e+33 eV-1 nan nan  False
	      e_0 3.000e+00 0.000e+00  TeV nan nan   True
	    alpha 2.859e+00 9.627e-02      nan nan  False
	 e_cutoff 1.757e+01 2.869e+00  TeV nan nan  False
	     beta 1.000e+00 0.000e+00      nan nan   True

Covariance: 

	   name   amplitude     e_0      alpha    e_cutoff     beta  
	--------- ---------- --------- --------- ---------- ---------
	amplitude  1.508e+66 0.000e+00 2.111e+30 -9.380e+32 0.000e+00
	      e_0  0.000e+00 0.000e+00 0.000e+00  0.000e+00 0.000e+00
	    alpha  2.111e+30 0.000e+00 9.269e-03  2.599e-01 0.000e+00
	 e_cutoff -9.380e+32 0.000e+00 2.599e-01  8.228e+00 0.000e+00
	     beta  0.000e+00 0.000e+00 0.000e+00  0.000e+00 0.000e+00 

Statistic: 33.454 (wstat)
Fit Range: [3.16227766e+07 1.99526231e+09] keV



In [10]:
model.compute_W()

<Quantity 9.02865044e+48 erg>